In [1]:
import pandas as pd

from ase import Atoms
import ase.io
from ase.formula import Formula

from pathlib import Path

import json

import pickle

from tqdm.notebook import tqdm

from mp_api.client import MPRester

from pymatgen.core.structure import Structure

import subprocess

In [4]:
v = {}
tm = {}

In [3]:
"""icsd:
[
    {
        'idnum': int,
        'icsd_col_code': int,
        'cif': cif:str,
        'ase.dict': cif:ase:dict,

        'stoichiometry.reduce': dict
    },
    ...
]

"""
with open('icsd.pickle','rb') as f:
    icsd = pickle.load(f)

In [6]:
def specie_nostructure_test(cif_str):
    try:
        s = Structure.from_str(cif_str, fmt='cif')
        for atom in s:
            atom.specie
    except (AttributeError, ValueError):
        return False
    return True

In [7]:
"""phosphors2
[
    {
        'formula': str,
        'stoichiometry': dict,
        'lambda': float,
    },
    ...
]
"""
with open('phosphors2.json','r') as f:
    phosphors2 = json.load(f)

In [8]:
"""phosphors2/3
[
    {
        'formula': str,
        'stoichiometry': dict,
        'lambda': float,

        'stoichiometry.reduce': dict,

        '[icsd]': [
            {
                'idnum': int,
                'icsd_col_code': int,
                'cif': cif:str,
                'ase.dict': cif:ase:dict,

                'stoichiometry.reduce': dict
            },
            ...
        ]
        'icsd': { ... },
    },
    ...
]
"""
for p in phosphors2:
    p['stoichiometry.reduce'] = Formula.from_dict(p['stoichiometry']).reduce()[0].count()

for p in tqdm(phosphors2):
    p['[icsd]'] = []
    for i in icsd:
        if p['stoichiometry.reduce'] == i['stoichiometry.reduce']:
            p['[icsd]'].append(i)

phosphors3 = []
for p in tqdm(phosphors2):
    try:
        v['[icsd]'] = p['[icsd]']
        v['[icsd]'].sort(key=lambda i: len(Atoms.fromdict(i['ase.dict'])))
        p['icsd'] = next(i for i in v['[icsd]'] if specie_nostructure_test(i['cif']))
        phosphors3.append(p)
    except:
        pass

  0%|          | 0/265 [00:00<?, ?it/s]

  0%|          | 0/265 [00:00<?, ?it/s]

/Users/xzhang1/opt/anaconda3/envs/gpt-cgcnn-export/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/xzhang1/opt/anaconda3/envs/gpt-cgcnn-export/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([1.0, 1.0, 1.0, 1.00001, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
  warnings.warn(msg)
/Users/xzhang1/opt/anaconda3/envs/gpt-cgcnn-export/lib/python3.11/site-packages/pymatgen/io/cif.py:1140: UserWarning: No structure parsed for 1 structure in CIF. Section of CIF file below.
  warnings.warn(f"No structure parsed for {i + 1} structure in CIF. Section of CIF file below.")
/Users/xzhang1/opt/anaconda3/envs/gpt-cgcnn-export/lib/python3.11/sit

In [10]:
l = []
for p in phosphors3:
    l.append({
        'idnum': p['icsd']['idnum'],
        'lambda': p['lambda']
    })
df = pd.DataFrame(l)
df.to_csv('cifs-icsd/id_prop.csv', index=False, header=False)

for p in tqdm(phosphors3):
    atoms = Atoms.fromdict(p['icsd']['ase.dict'])
    atoms.write('scratch/POSCAR', format='vasp')
    subprocess.run(['phonopy --symmetry -c POSCAR'], cwd='scratch', shell=True)
    atoms = ase.io.read('scratch/PPOSCAR', format='vasp')
    atoms.write(f"cifs-icsd/{p['icsd']['idnum']}.cif", format="cif")

phonopy_version: '2.20.0'
space_group_type: 'Pm-3m'
space_group_number: 221
point_group_type: 'm-3m'
space_group_operations:
- rotation: # 1
  - [ 1,  0 , 0]
  - [ 0,  1 , 0]
  - [ 0,  0 , 1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 2
  - [-1,  0 , 0]
  - [ 0, -1 , 0]
  - [ 0,  0 ,-1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 3
  - [ 0, -1 , 0]
  - [ 1,  0 , 0]
  - [ 0,  0 , 1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 4
  - [ 0,  1 , 0]
  - [-1,  0 , 0]
  - [ 0,  0 ,-1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 5
  - [-1,  0 , 0]
  - [ 0, -1 , 0]
  - [ 0,  0 , 1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 6
  - [ 1,  0 , 0]
  - [ 0,  1 , 0]
  - [ 0,  0 ,-1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 7
  - [ 0,  1 , 0]
  - [-1,  0 , 0]
  - [ 0,  0 , 1]
  translation: [ 0.00000,  0.00000,  0.00000 ]
- rotation: # 8
  - [ 0, -1 , 0]
  - [ 1,  0 , 0]
  - [ 0,  0 ,-1]
  translat

In [ ]:
"""phosphors4
[
    {
        'formula': 'Ba2Mg(PO4)2:Eu2+',
        'stoichiometry': {'Ba': 2, 'Mg': 1, 'P': 2, 'O': 8, 'Eu': 1},
        'lambda': 450,
        'mpid': 'mp-1009000',
        'structure': Structure
    },
    ...
]
"""

with open('phosphors2.json','r') as f:
    phosphors2 = json.load(f)

phosphors4 = []
with MPRester('YOUR_MP_API_KEY_HERE') as mpr:
    for p in tqdm(phosphors2):
        docs = mpr.summary.search(
            formula = Formula.from_dict(p['stoichiometry']).format('metal'),
            fields = ['material_id']
        )
        if len(docs) != 0:
            mpid = docs[0].material_id
            structure = mpr.materials.get_structure_by_material_id(mpid)
            phosphors4.append({
                'formula': p['formula'],
                'stoichiometry': p['stoichiometry'],
                'lambda': p['lambda'],
                'mpid': mpid,
                'structure': structure
            })

l = []
for p in phosphors4:
    l.append({
        'id': p['mpid'],
        'lambda': p['lambda'],
    })
pd.DataFrame(l).to_csv('cifs-mp/id_prop.csv', index=False, header=False)

id_prop = []
for p in phosphors4:
    p['structure'].to(filename=f"cifs-mp/{p['mpid']}.cif")